<a href="https://colab.research.google.com/github/viniciusdutra314/CanalSingularidade/blob/main/WhisperAi/Whisper%2BDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🗣️ **Whisper** - *Notebook criado por [DotCSV](https://www.youtube.com/channel/UCy5znSnfMsDwaLlROnZ7Qbg) e traduzido pelo Singularidade*

🚨 Deixe ativado a **Aceleração por Hardware** com GPU `"Ambiente de Execução" > "Alterar o tipo de ambiente de execução"`

In [ ]:
#@title ⚙️ Execute para instalar os módulos necessários.
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from google.colab import drive
drive.mount('/content/drive')
videos=[]
def download_file(j):
  yt = YouTube(j)
  title=yt.title
  title = unidecode(title) #removendo caracteres especiais e convertando para mp3
  title = title.translate(title.maketrans('','',string.punctuation))
  title= title.lower() +".mp3"
  drivepath="/content/drive/MyDrive/legendasyoutube/"+title
  audio_stream = yt.streams.filter(only_audio=True).first()
  audio_stream.download(filename=title)
  video_stream = yt.streams.filter(progressive=True).order_by("resolution").desc().first()
  video_stream.download(drivepath)
  videos.append("'"+title+"'")

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install pytube
!pip install unidecode
!pip install ffmpeg

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

## Youtube


In [17]:
from pytube import YouTube
from unidecode import unidecode
import string
import subprocess
# URL do vídeo, dentro do colchete você pode colocar quantas urls quiser
#só lembre de colocar entre aspas("url" ou 'url') e separadas por virgula (,)
mode="transcribe" ; quality="large"
url = ['https://www.youtube.com/watch?v=RzEjqJHW-NU'] ;
for j in url: download_file(j)
for k in videos:
  if mode=="transcribe":
    command=f"whisper {k} --task transcribe --model {quality} --verbose False --output_format srt --output_dir {'/content/drive/MyDrive/legendasyoutube/'+k}\n"
  elif mode=="translate":
    command=f"!whisper {k} --task translate --model {quality} --verbose False --output_dir {'/content/drive/MyDrive/legendasyoutube/'+k}\n"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

for line in process.stdout:
  print(line, end='')


Detected language: Spanish

100%|██████████| 47266/47266 [01:40<00:00, 469.92frames/s]


In [ ]:
audios=[]
diretorio="/content/"
mode="transcribe" ; quality="large"
audios=[diretorio+j +".mp3" for j in audios]

for k in audios:
  if mode=="transcribe":
    command=f"whisper {k} --task transcribe --model {quality} --verbose False --output_format srt --output_dir {'/content/drive/MyDrive/legendasyoutube/'+k}\n"
  elif mode=="translate":
    command=f"!whisper {k} --task translate --model {quality} --verbose False --output_dir {'/content/drive/MyDrive/legendasyoutube/'+k}\n"

In [ ]:
!whisper --help